In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("StockData.csv")
df2 = pd.read_csv("CleanedNewData.csv")
df2 = df2.drop('Unnamed: 0', axis=1)

In [5]:
#new data frame to us with df2 to determine what article were posted on non-business days.
df["buissnes day"] = 1
dfb = df[["buissnes day","Date"]]

In [17]:
#make new column to reliably merge the two datasets on
dfb["merge"] = pd.to_datetime(dfb["Date"])
df2["merge"] =  pd.to_datetime(df2["Time"])

<ipython-input-17-63ddd7fd5878>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb["merge"] = pd.to_datetime(dfb["Date"])


In [11]:
#merge dataset
result = pd.merge(df2, dfb, how='left', on=['merge'])

In [12]:
result = result.fillna(0)
temp0 = result

Funtions developed that combines rows of news data that take place on non buissness days. 

In [13]:
def Eliminalterows(NewsX):
    indexs = []
    new = []
    for index, row in NewsX.iterrows():
        x = NewsX.loc[index,"buissnes day"]
        temp = index + 1
        if x == 0:
            indexs.append(index)
            indexs.append(temp)
            hold = []
            count = 0
            for (columnName, columnData) in NewsX.loc[index].iteritems():
                if columnName == "Time" or columnName == "buissnes day" or columnName == "Date" or columnName == "merge" :
                    data = NewsX.loc[temp,columnName]
                    hold.append(data)
                else:
                    X = columnData
                    Y = NewsX.loc[temp,columnName]
                    val1 = X + Y

                    val2 = val1/2
                    if count > 12:
                        X = columnData
                        Y = NewsX.loc[temp,columnName]

                        val1 = X + Y
                        val2 = int(val1)/2
                        if val2 != 0:
                            val2 = 1.0
                            hold.append(val2)
                        else:
                            hold.append(val2)                    
                    else:
                        hold.append(val2)
                count = count + 1
            new.append(hold)


        if temp == NewsX.shape[0] - 1:
            break
    return new, indexs 
def removeduprows(df):    
    new, index = Eliminalterows(df)
    dups = len(index)
    print("dup row count:" , dups)
    new1 = pd.DataFrame(new, columns = df.columns)
    out = df.drop(index)
    out = pd.concat([out, new1], axis=0, sort=False)
    out = out.sort_values(by=['merge'])
    out = out.reset_index(drop=True)
    return out

In [14]:
out = removeduprows(temp0)
out2 = removeduprows(out)
out3 = removeduprows(out2)
out4 = removeduprows(out3)

dup row count: 324
dup row count: 48
dup row count: 4
dup row count: 0


In [24]:
#clean final output for the completed news data
BisDayNews = out4.drop(['buissnes day','Date', 'merge'], axis=1)
BisDayNews = out4.drop('Time', axis=1)

In [ ]:
BisDayNews.to_csv('BisDayNews.csv')